# IBM Applied Data Science Capstone Project's Notebook by Kurniayazid

## Week 3 - Part 1 

In [1]:
# importing libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import urllib.request

In [2]:
# Targeted website's source
source = requests.get('https://en.m.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [3]:
# Scraping source using bs4
soup = BeautifulSoup(source, 'lxml')

In [4]:
# Finding all table
table = soup.find_all('table')

In [5]:
# Finding targeted table
target_table = soup.find('table', class_='wikitable sortable')

In [6]:
# Loop the data through the rows -- put the data into list
post = []
borough = []
neighborhood = []

for x in target_table.findAll('tr'):
  cells = x.findAll('td')
  if len(cells)==3:
    post.append(cells[0].find(text=True))
    borough.append(cells[1].find(text=True))
    neighborhood.append(cells[2].find(text=True))

In [7]:
# Put the data into dataframe
df = pd.DataFrame(post,columns=['postcode'])
df['borough'] = borough
df['neighborhood'] = neighborhood
df.head(12)

,postcode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [8]:
# Adjust neighborhood if postcode is M5A -- I think because the data has been updated
df.neighborhood.loc[df.postcode=='M5A'] = 'Regent Park, Harbourforont'
df[df.postcode=='M5A']

,postcode,borough,neighborhood
4,M5A,Downtown Toronto,"Regent Park, Harbourforont"


In [9]:
# Drop if borough is 'Not assigned'
df = df[df.borough!='Not assigned']

In [10]:
# Remove \n in neighborhood and borough
df['neighborhood'] = df['neighborhood'].str.replace('\n','')
df['borough'] = df['borough'].str.replace('\n','')

In [11]:
# Dataframe shape
df.shape

(210, 3)

In [12]:
# List of data
df.head(12)

,postcode,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourforont"
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


## Week 3 - Part 2 

In [13]:
# Import Latitude and Longitude data for the respected Postal code
geocode = pd.read_csv('http://cocl.us/Geospatial_data')
geocode.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
# Rename primary key (postal code to postcode)
geocode.rename(columns={'Postal Code': 'postcode'},inplace=True)

In [15]:
# Merge the geocode data to the dataframe
dg = pd.merge(df,geocode,on='postcode',how='inner')

In [16]:
# Summary Statistics for the data
dg.describe(include='all')

,postcode,borough,neighborhood,Latitude,Longitude
count,210,210,210,210.000000,210.000000
unique,103,10,208,NaN,NaN
top,M8Y,Etobicoke,St. James Town,NaN,NaN
freq,8,45,2,NaN,NaN
mean,NaN,NaN,NaN,43.697600,-79.409866
std,NaN,NaN,NaN,0.054080,0.103862
min,NaN,NaN,NaN,43.602414,-79.615819
25%,NaN,NaN,NaN,43.650943,-79.494292
50%,NaN,NaN,NaN,43.689300,-79.400049
75%,NaN,NaN,NaN,43.739416,-79.347260


In [17]:
# Check the top 12 data
dg.head(12)

,postcode,borough,neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourforont",43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188


## Week 3 - Part 3 

In [18]:
!pip install folium==0.5.0

In [19]:
import folium # Map plotting library
from tqdm import tqdm # Loading time
from collections import deque
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

In [41]:
# CLIENT ID and CLIENT SECRET
CLIENT_ID = 'SECRET' # your Foursquare ID
CLIENT_SECRET = 'SECRET' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SECRET
CLIENT_SECRET:SECRET


In [21]:
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(dg['Latitude'], dg['Longitude'], dg['borough'], dg['neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in tqdm(zip(names, latitudes, longitudes), total = names.size):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
Toronto_venues = getNearbyVenues(dg.neighborhood,
                            dg.Latitude,
                            dg.Longitude)

100%|██████████| 210/210 [00:45<00:00,  5.57it/s]


In [24]:
print(Toronto_venues.shape)
Toronto_venues.head()

(4355, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [25]:
Toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
St. James Town          145
Adelaide                100
First Canadian Place    100
Richmond                100
King                    100
Name: Venue, dtype: int64

In [26]:
# Total unique categories
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 267 uniques categories.


## One Hot Encoding

In [27]:
Toronto_ohe = pd.get_dummies(Toronto_venues["Venue Category"],
                             prefix = "",
                             prefix_sep = "")

Toronto_ohe["Neighborhood"] = Toronto_venues["Neighborhood"]


nindex = list(Toronto_ohe.columns).index("Neighborhood")
cols = deque(Toronto_ohe.columns)
cols.rotate(-nindex)
cols = list(cols)
Toronto_ohe = Toronto_ohe[cols]

Toronto_ohe.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
print('This is the toronto one hot encoding dimension', Toronto_ohe.shape)

This is the toronto one hot encoding dimension (4355, 267)


In [29]:
Toronto_grouped = Toronto_ohe.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,...,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon
0,Adelaide,0.01,0.0,0.01,0.01,0.01,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
2,Agincourt North,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
3,Albion Gardens,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0
4,Alderwood,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0


## Most frequent venue per neighborhood

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Restaurant,Thai Restaurant,Café,Sushi Restaurant,Steakhouse,Bar,Gastropub,Vegetarian / Vegan Restaurant,Lounge
1,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Nail Salon,Asian Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store
2,Agincourt North,Playground,Park,Nail Salon,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports
3,Albion Gardens,Grocery Store,Discount Store,Beer Store,Fast Food Restaurant,Japanese Restaurant,Sandwich Place,Fried Chicken Joint,Pizza Place,Pharmacy,Aquarium
4,Alderwood,Pizza Place,Skating Rink,Sandwich Place,Pool,Gym,Pharmacy,Pub,Coffee Shop,Bagel Shop,Baby Store


## Neighborhood clustering

In [32]:
from sklearn.decomposition import PCA # Using PCA to increase clustering efficiency
pca = PCA(.95)
Toronto_grouped_clustering = pca.fit_transform(Toronto_grouped.drop('Neighborhood', 1))
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

In [33]:
# Clusters k=5
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(kmeans.labels_.shape)

[0 0 3 0 0 0 0 0 0 0]
(201,)


In [34]:
Toronto_grouped["Cluster Labels"] = kmeans.labels_

# add clustering labels
Toronto_combined = dg.merge(Toronto_grouped, left_on = "neighborhood", right_on = "Neighborhood", how = "outer")

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_combined = Toronto_combined.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_combined["Cluster Labels"] = Toronto_combined["Cluster Labels"].fillna(5).astype("int")

Toronto_combined.head() # check the last columns!

,postcode,borough,neighborhood,Latitude,Longitude,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,Parkwoods,0.0,0.0,0.0,0.0,...,Food & Drink Shop,Park,Nail Salon,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,Victoria Village,0.0,0.0,0.0,0.0,...,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Arts & Crafts Store,American Restaurant,Antique Shop,Aquarium,Art Gallery
2,M5A,Downtown Toronto,"Regent Park, Harbourforont",43.654260,-79.360636,"Regent Park, Harbourforont",0.0,0.0,0.0,0.0,...,Coffee Shop,Café,Pub,Bakery,Park,Restaurant,Breakfast Spot,Theater,Mexican Restaurant,Cosmetics Shop
3,M6A,North York,Lawrence Heights,43.718518,-79.464763,Lawrence Heights,0.0,0.0,0.0,0.0,...,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Miscellaneous Shop,Women's Store,Event Space
4,M6A,North York,Lawrence Manor,43.718518,-79.464763,Lawrence Manor,0.0,0.0,0.0,0.0,...,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Miscellaneous Shop,Women's Store,Event Space


## Mapping cluster

In [35]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

kclusters = kclusters + 1

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_combined['Latitude'],
                                  Toronto_combined['Longitude'],
                                  Toronto_combined['Neighborhood'],
                                  Toronto_combined['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyzing cluster

### Cluster-1

In [36]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 0,
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Arts & Crafts Store,American Restaurant,Antique Shop,Aquarium,Art Gallery
2,Coffee Shop,Café,Pub,Bakery,Park,Restaurant,Breakfast Spot,Theater,Mexican Restaurant,Cosmetics Shop
3,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Miscellaneous Shop,Women's Store,Event Space
4,Clothing Store,Furniture / Home Store,Boutique,Accessories Store,Vietnamese Restaurant,Gift Shop,Coffee Shop,Miscellaneous Shop,Women's Store,Event Space
5,Coffee Shop,Park,Café,Discount Store,Beer Bar,Japanese Restaurant,Italian Restaurant,Sandwich Place,Creperie,Seafood Restaurant


**Cluster-1** is dominated by coffeeshop, café, pub, bar, to restaurant

### Cluster-2

In [37]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 1, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
106,Baseball Field,Airport Terminal,Bar,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports
107,Baseball Field,Airport Terminal,Bar,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports
197,Locksmith,Baseball Field,Nail Salon,Asian Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Athletics & Sports
198,Locksmith,Baseball Field,Nail Salon,Asian Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Athletics & Sports
199,Locksmith,Baseball Field,Nail Salon,Asian Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Athletics & Sports


**Cluster-2** is dominated by baseball field, locksmith, bank, and airport terminal

### Cluster-3

In [38]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 2, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Fast Food Restaurant,Nail Salon,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports
8,Fast Food Restaurant,Nail Salon,Airport Terminal,Bank,Bakery,Bagel Shop,Baby Store,BBQ Joint,Auto Workshop,Athletics & Sports


**Cluster-3** is dominated by fast food restaurant and nail salon

### Cluster-4

In [39]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 3, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Food & Drink Shop,Park,Nail Salon,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
38,Park,Women's Store,Market,Nail Salon,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
60,Park,Convenience Store,Metro Station,Nail Salon,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
70,Construction & Landscaping,Airport,Park,Nail Salon,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
71,Construction & Landscaping,Airport,Park,Nail Salon,Airport Terminal,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant


**Cluster-2** is dominated by park, and shop

### Cluster-5

In [40]:
Toronto_combined.loc[Toronto_combined['Cluster Labels'] == 4, 
                     "1st Most Common Venue":"10th Most Common Venue"].head()

,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Discount Store,Convenience Store,Department Store,Hobby Shop,Coffee Shop,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
67,Discount Store,Convenience Store,Department Store,Hobby Shop,Coffee Shop,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
68,Discount Store,Convenience Store,Department Store,Hobby Shop,Coffee Shop,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant
102,Convenience Store,Discount Store,Sandwich Place,Coffee Shop,Restaurant,Nail Salon,Athletics & Sports,Aquarium,Art Gallery,Arts & Crafts Store
103,Convenience Store,Discount Store,Sandwich Place,Coffee Shop,Restaurant,Nail Salon,Athletics & Sports,Aquarium,Art Gallery,Arts & Crafts Store


**Cluster-5** is dominated by discount store, convenience store, and department store